In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
from folium import IFrame
from folium import Map, FeatureGroup, Marker, LayerControl, Popup


In [ ]:
# read in CSV from webscrape
cummins_df = pd.read_csv('/home/nhac/Documents/NSS/Python/capstone/cummins_scrape_1.csv')
cummins_df.head()

In [ ]:
# explore loop strategy
for i in range(1, 4):
    for j in range(2, 5):
        print('zone' + str(i) + 'zone' + str(j))

In [ ]:
# more explore loop strategy
empty_list = []
for i in range(3, 10):
    for j in ['Apple','Peach','Plum','Pear','Nectarine','Cherry','Apricot']:       
        globals()[j + str(i)] = []
        

In [ ]:
# test for results calling column and copying 
poop = cummins_df[['Apple3']].copy()
poop

In [ ]:
# Build loop to grab columns and data from cummins_df and output to dataframe

fruit_list = pd.DataFrame()     # create empty dataframe to append results
for i in range(3,10):     # iterate through numbers to assign for zone
    for j in ['Apple','Peach','Plum','Pear','Nectarine','Cherry','Apricot']:     # for every fruit type names
        col_name = j + str(i)     # concat zone number to fruit type name to get matching column name in cummins_df (Apple3, Apple4, etc.)
        tmp_f_list = cummins_df[[col_name]].copy()     # tmp list with column name matching column name (variable)
        tmp_f_list = tmp_f_list[tmp_f_list[col_name].notna()]     # drop the NaNs from the list for later list concat
        tmp_f_list = tmp_f_list.rename(columns={col_name:'fruit_name'})     # rename the variable column name, with static name
        tmp_f_list['zone'] = i     # add a column to the tmp DF and populate the column with variable RANGE number
        tmp_f_list[['type']] = j     # add a column to the tmp DF and populate with variable fruit type name
        fruit_list = pd.concat([fruit_list, tmp_f_list])   # append results from tmp_f_list to previously empty df, fruit_list

In [ ]:
# reset index
fruit_list = fruit_list.reset_index(drop=True)
fruit_list.head()


In [ ]:
print('Success!')

In [ ]:
# to back up/flat file CSV
#fruit_list.to_csv('fruit_list.csv')

In [ ]:
# read in hardiness zones geojson
geo_h_zones = gpd.read_file('../data/us-hardiness-zones_1291.geojson')

In [ ]:
# normalize column names for merge
geo_h_zones = geo_h_zones.rename(columns={'name':'zone'})

In [ ]:
# CRS to EPSG
geo_h_zones = geo_h_zones.to_crs(epsg=4326)

In [ ]:
# check for GeoDataFrame
type(geo_h_zones)

In [ ]:
# convert polys to centroids from geojson
centroid_df = geo_h_zones.geometry.centroid.to_frame(name='centroid')

In [ ]:
# join geojson with newly created centroids
geo_df = pd.concat([geo_h_zones, centroid_df] , axis=1)

In [ ]:
# normalize 'zone' column type to int for merge
geo_df.zone = geo_df.zone.astype(int)

In [ ]:
# merge fruit data with geo data on 'zone'
all_frame = pd.merge(fruit_list, geo_df, how='inner', on='zone')

In [ ]:
# Extract lat and lon from centroid, insert into new columns
all_frame['lon'] = all_frame.centroid.apply(lambda p: p.x)
all_frame['lat'] = all_frame.centroid.apply(lambda p: p.y)

In [ ]:
# View for reference
#all_frame

In [ ]:
# Vew for reference
#all_frame.dtypes

In [ ]:
all_frame.centroid = all_frame.centroid.astype(str)

In [ ]:
z3 = all_frame[all_frame['zone'] == 3]

In [ ]:
# View for reference
#geo_h_zones

In [ ]:
maps_fruit_list = all_frame.drop_duplicates(subset=['fruit_name','zone','type'])

In [ ]:
choro_count = maps_fruit_list.groupby('zone').count().reset_index()

In [ ]:
choro_count.zone = choro_count.zone.astype(str)

In [ ]:
choro_count = pd.merge(geo_h_zones, choro_count, how='inner', on='zone')

In [ ]:
# View for reference
#choro_count

In [ ]:
geometry = geo_h_zones['geometry']
simplified_geometry = geometry.simplify(tolerance=0.125)
geo_h_zones['geometry'] = simplified_geometry

In [ ]:
# Create a Folium map
 
m = folium.Map(location=(31.51073, -96.4247), 
               zoom_start=4, 
               tiles='openstreetmap', 
               width="100%", 
               height="100%",
               no_wrap=True,
               min_zoom=4,
               max_zoom=6
              )

# Add the polygons to the map (maybe redundant)
geojson = geo_h_zones.to_crs(epsg=4326).to_json()

# add chloropleth layer
folium.Choropleth(
    geo_data=geojson,
    name='Hardiness Zones',
    data=choro_count,
    columns=['zone','fruit_name'],
    fill_color='YlGn',
    key_on='feature.properties.zone',
    legend_name='Total Number of Fruit Types'
).add_to(m)

folium.Choropleth(
    geo_data="data2.json",
    data="data2.csv",
    columns=["region", "value"],
    key_on
# Create a FeatureGroup object
#fg = folium.map.FeatureGroup()

# Iterate through the "zone" column of the data frame
for z in maps_fruit_list['zone']:
    # Select rows that have the current category
    items = maps_fruit_list[maps_fruit_list['zone'] == z]
    
    # Place the items in an HTML table
    html_table = "<table>"
    for i, row in items.iterrows():
        html_table += "<tr><td>{}</td><td>{}</td><td>{}</td></tr>".format(row['fruit_name'], row['type'], row['zone'])
    html_table += "</table>"

    # Add a marker to the map for the zone centroid    
    marker = folium.Marker(location=[row['lat'], row['lon']],
                           popup=folium.Popup(html_table, 
                                              max_height=300, 
                                              scroll_wheel=True, 
                                              clustered_marker=True),
                          text="wtf")
    marker.add_to(m)

folium.LayerControl().add_to(m)


In [ ]:
m